In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from itertools import combinations
import matplotlib.pyplot as plt # importing matplotlib
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
import statistics

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
# subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_with_path = ["" for i in range(3)]
subfolder_with_path[0] = main_directory + '/'+'ABAcluster_cropped' # cluster files with samples in columns to calulate volumes
subfolder_with_path[1] = main_directory + '/'+'ABAconsensus_cropped' # activity files
subfolder_with_path[2] = main_directory + '/'+'zzz_input_and_reference_files'

In [5]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"SAMPLE": int,
                                      "hemisphere": "string",
                                      "marker": "string",
                                      "TX_GROUP": "string", 
                                      "context":"string"})

# Reading "gubra_intensities_template.csv" file
gubra_intensities_template = pd.read_csv("gubra_intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "LabelAbrv": "string"})
# Reading "brain_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map_fixed.csv", 
                               dtype={"IDPath": "string", "LabelAbrv": "string",
                                      "raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [6]:
#Saving drugs and markers' names from the "sample_overview" file

drugs = samples_overview['TX_GROUP'].unique()
markers = samples_overview['marker'].unique()

d = 0
m = 0
counters_names = []
while (m < len(markers)):
    while (d < len(drugs)):
        counters_names.append(drugs[d] + '_' + markers[m])
        d += 1
    m += 1
    d = 0

In [17]:
# Changing directory to the "ABAcluster_cropped" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "ABAcluster_cropped" folder to obtain number of clusters 
# and counting the number of columns in each file to count the number of samples

clusters = os.listdir(subfolder_with_path[0]) 
number_of_clusters = len(clusters)




In [19]:
 samples = pd.read_csv(list[0]).columns


In [22]:
samples_and_clusters_matrix = ["" for x in range(number_of_clusters)]


samples_and_clusters_matrix

['', '']